In [1]:
!pip install transformers datasets

In [2]:
import pandas as pd
import torch
from transformers import (
    DistilBertTokenizer,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments,
    pipeline
)
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')


Using device: cuda


In [4]:
# Load your dataset
df = pd.read_csv('tweet_sent_train.csv')

# Display the first few rows to understand the structure
print(df.head())


   Unnamed: 0 Sentiment                                            Context
0           0  Positive  I am coming to the borders and I will kill you...
1           1  Positive  im getting on borderlands and i will kill you ...
2           2  Positive  im coming on borderlands and i will murder you...
3           3  Positive  im getting on borderlands 2 and i will murder ...
4           4  Positive  im getting into borderlands and i can murder y...


In [5]:
# Rename columns (replace with your actual column names)
df = df.rename(columns={'Context': 'text', 'Sentiment': 'label'})

# Drop rows with missing values
df = df.dropna(subset=['text', 'label'])

# Remove duplicates
df = df.drop_duplicates(subset=['text'])


In [6]:
# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform labels
df['label'] = label_encoder.fit_transform(df['label'])

# Save the mapping to understand the encoding
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_mapping)


Label Mapping: {'Irrelevant': 0, 'Negative': 1, 'Neutral': 2, 'Positive': 3}


In [7]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


In [8]:
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))


In [9]:
# Load tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Define the tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

# Tokenize the datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/55592 [00:00<?, ? examples/s]

Map:   0%|          | 0/13898 [00:00<?, ? examples/s]

In [10]:
# Set format to PyTorch tensors
tokenized_train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
tokenized_test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


In [11]:
# Load the model
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=len(label_encoder.classes_)
)

# Move the model to GPU
model.to(device)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [12]:
training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    evaluation_strategy='epoch',     # Evaluate every epoch
    save_strategy='epoch',           # Save checkpoint every epoch
    learning_rate=2e-5,              # Learning rate
    per_device_train_batch_size=16,  # Batch size per device during training
    per_device_eval_batch_size=16,   # Batch size per device during evaluation
    num_train_epochs=3,              # Total number of training epochs
    weight_decay=0.01,               # Weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=10,                # Log every 10 steps
    load_best_model_at_end=True,     # Load the best model at the end
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='weighted'
    )
    acc = accuracy_score(labels, predictions)
    return {'accuracy': acc, 'precision': precision, 'recall': recall, 'f1': f1}


In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [17]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.158300,0.429530,0.920420,0.921578,0.920420,0.920248
2,0.041500,0.403204,0.936538,0.936875,0.936538,0.936531
3,0.003000,0.381289,0.944884,0.944888,0.944884,0.944884


TrainOutput(global_step=10425, training_loss=0.07014152338480403, metrics={'train_runtime': 2028.0303, 'train_samples_per_second': 82.235, 'train_steps_per_second': 5.14, 'total_flos': 5523292712706048.0, 'train_loss': 0.07014152338480403, 'epoch': 3.0})

In [18]:
trainer.evaluate()


{'eval_loss': 0.3812892735004425,
 'eval_accuracy': 0.9448841559936682,
 'eval_precision': 0.9448876227844527,
 'eval_recall': 0.9448841559936682,
 'eval_f1': 0.9448838655076927,
 'eval_runtime': 52.9645,
 'eval_samples_per_second': 262.402,
 'eval_steps_per_second': 16.407,
 'epoch': 3.0}

In [19]:
model.save_pretrained('./fine_tuned_distilbert')
tokenizer.save_pretrained('./fine_tuned_distilbert')


('./fine_tuned_distilbert/tokenizer_config.json',
 './fine_tuned_distilbert/special_tokens_map.json',
 './fine_tuned_distilbert/vocab.txt',
 './fine_tuned_distilbert/added_tokens.json')

In [20]:
# Load the fine-tuned model and tokenizer
sentiment_pipeline = pipeline(
    'text-classification',
    model='./fine_tuned_distilbert',
    tokenizer='./fine_tuned_distilbert',
    device=0 if torch.cuda.is_available() else -1  # Use GPU if available
)


In [21]:
# Example texts
texts = ["I love this product!", "This is the worst experience I've ever had."]

# Get predictions
predictions = sentiment_pipeline(texts)

# Decode labels back to original
for text, pred in zip(texts, predictions):
    label_id = int(pred['label'].split('_')[-1])
    label_name = label_encoder.inverse_transform([label_id])[0]
    print(f"Text: {text}\nPredicted Sentiment: {label_name}\n")


Text: I love this product!
Predicted Sentiment: Positive

Text: This is the worst experience I've ever had.
Predicted Sentiment: Negative

